# Utility for converting .csv from ANSYS to OpenFOAM Field Files
Before running this, you do the following first:
1. Convert the ANSYS .msh file to an OpenFOAM polyMesh using <pre>fluent3DmeshToFoam ANSYS_MESH.msh</pre>
2. Use OpenFOAM utility or otherwise, write the cell centre coordinates to the 0 directory i.e. CellPos


In [1]:
# File for parsing .csv file as output from Fluent
import pandas as pd
import numpy as np
import re

def ReadVectorFieldValues(CasePath,TimeStr,FieldName):
    filePath = CasePath + TimeStr + "/" + FieldName
#     Open file and read and split the lines
    with open(filePath) as f:
        lines = f.read().splitlines() 
    i = 0
    for line in lines:
        if lines[i] == "(":
            nValues = int(lines[i-1]);
#             print(nValues)
            iStart = i+1;
            
        if lines[i] == "boundaryField":
            iBoundary = i
            iEnd = i-4;
        i = i+1
#     print(iStart,lines[iStart])
#     print(iEnd,lines[iEnd])

    boundaryLines = lines[iBoundary:-1]
    boundaryStr = ""
    b = 0
    for B in boundaryLines:
        boundaryLines[b] = re.sub("calculated","zeroGradient",B)
        if "value" in B:
            boundaryLines[b]=""
        boundaryStr = boundaryStr + str(boundaryLines[b]) + str("\n")
        b = b+1
    
    boundaryStr = str(boundaryStr)
    
    lines = lines[iStart:iEnd+1]
    

#     print(len(lines))
    i =0
    for l in lines:
        lines[i] = lines[i].strip('(')
        lines[i] = lines[i].strip(')')
        lines[i] = lines[i].split()

        j=0
        for c in lines[i]:
            lines[i][j] = float(c)
            j=j+1
        
        i = i+1
    lines = np.array(lines)
    
    return lines, boundaryStr

# lines, b = ReadVectorFieldValues(CasePath,"0","CellPos")
# print(b)

def  writeField(ValueType,CasePath,TimeStr,FieldName,FieldValues,boundaryInfoStr):
    buffer=[]
    buffer.append('/*--------------------------------*- C++ -*----------------------------------*\\')
    buffer.append('|=========                 |                                                 |')
    buffer.append('| \\      /  F ield         | foam-extend: Open Source CFD                    |')
    buffer.append('|  \\    /   O peration     | Version:     4.0                                |')
    buffer.append('|   \\  /    A nd           | Web:         http://www.foam-extend.org         |')
    buffer.append('|    \\/     M anipulation  |                                                 |')
    buffer.append('\*---------------------------------------------------------------------------*/')
    buffer.append('//             THIS FILE WAS CREATED BY D. DREELAN IN MATLAB')
    buffer.append('FoamFile')
    buffer.append('{')
    buffer.append('    version     2.0;')
    buffer.append('    format      ascii;')
#     iStart = len(buffer)
#     i = iStart + 1;

    if ValueType == 'vector':
        buffer.append('    class       volVectorField;')
    elif ValueType == 'scalar':
        buffer.append('    class       volScalarField;')
#     else: # Need to quit out and say not recognised

    buffer.append('    location   "' + TimeStr + '";')
    buffer.append('    object    ' + FieldName + ';')
    buffer.append('}')
    if FieldName == "T":
        buffer.append('dimensions      [0 0 0 1 0 0 0];')
    else:
        buffer.append('dimensions      [0 0 0 0 0 0 0];')
    buffer.append('internalField   nonuniform List<'+str(ValueType)+'>')
    buffer.append(str(len(FieldValues)))
    buffer.append('(')

# %      ADD ALL FIELD VALUES
    if ValueType == 'scalar':
        for v in range(0,len(FieldValues)):
            buffer.append(str(FieldValues[v]))

    elif ValueType == 'vector':
        for v in range(0,len(FieldValues)):
            buffer.append("(" +str(FieldValues[v,0])+" " +str(FieldValues[v,1])+" " +str(FieldValues[v,2])+")")
            
    buffer.append(")")
    buffer.append(";")
    
    buffer.append(boundaryInfoStr)
    
#     Add boundary info
#     buffer.append('{')
#     buffer.append('frontAndBack{ type            zeroGradient;}')
#     buffer.append('left{ type            zeroGradient;}' )
#     buffer.append('bottom{ type            zeroGradient;}' )
#     buffer.append('right{ type            zeroGradient;}' )
#     buffer.append('top{ type            zeroGradient;}' )
#     buffer.append('}' )
#     buffer.append(" ")
#     buffer.append(" " )
    
#     Buffer is complete, need to print to file
    pathToFile = CasePath + TimeStr + "/" + FieldName
    
    f = open(pathToFile, "a")
    f.truncate()
    for line in buffer:
        f.write(line + "\n")
    f.close()

In [2]:
def printdfMeshInfo(df):
    uniqueX = sorted(df["x-coordinate"].unique())
    uniqueY = sorted(df["y-coordinate"].unique())
    uniqueZ = sorted(df["z-coordinate"].unique())

    print("---------- CSV MESH INFORMATION ----------")
    print("Mesh number of cells. x: " + str(len(uniqueX)) + "\ty: " + str(len(uniqueY)) + "\tz: " + str(len(uniqueZ)) + "\tTotal: " + str(len(df)))
    print("CSV Mesh x-dims: ",min(df["x-coordinate"]), max(df["x-coordinate"]))
    print("CSV Mesh y-dims: ",min(df["y-coordinate"]), max(df["y-coordinate"]))
    print("CSV Mesh z-dims: ",min(df["z-coordinate"]), max(df["z-coordinate"]))
    print("------------------------------------------")
    
# Get max dimensions from the CellCentre that has just been read
def printCellCenMeshInfo(CellCentre):
    xMax = 0.0
    yMax = 0.0
    zMax = 0.0
    xMin = 1e12
    yMin = 1e12
    zMin = 1e12
    for c in CellCentre:
        if c[0] > xMax: xMax = c[0]
        if c[1] > yMax: yMax = c[1]
        if c[2] > zMax: zMax = c[2]
        if c[0] < xMin: xMin = c[0]
        if c[1] < yMin: yMin = c[1]
        if c[2] < zMin: zMin = c[2]
    
    print("---------- CellCentre MESH INFORMATION ----------")
    print("Number of cells: ", len(CellCentre))
    print("CSV Mesh x-dims: ",xMin, xMax)
    print("CSV Mesh y-dims: ",yMin, xMax)
    print("CSV Mesh z-dims: ",zMin, xMax)
    print("------------------------------------------")
    
def defineMeshNative(df):
    # Get unique and sorted lists of coordinates
    uniqueX = sorted(df["x-coordinate"].unique())
    uniqueY = sorted(df["y-coordinate"].unique())
    uniqueZ = sorted(df["z-coordinate"].unique())

    print("Mesh number of cells. x: " + str(len(uniqueX)) + "\ty: " + str(len(uniqueY)) + "\tz: " + str(len(uniqueZ)))

    # Mesh is defined by starting with lowest y and z

    ix = 0
    iy = 0
    iz = 0

    # uniqueX = [1,2,3,4,5]
    # uniqueY = [1,2,3,4,5]
    # uniqueZ = [1,2,3,4,5]

    totalCells = len(uniqueX)*len(uniqueY)*len(uniqueZ)

    CellCentre = np.zeros((totalCells,3))

    iTotal = 0
    for z in uniqueZ:
        for y in uniqueY:
            for x in uniqueX:
                CellCentre[iTotal][0] = uniqueX[ix]
                CellCentre[iTotal][1] = uniqueY[iy]
                CellCentre[iTotal][2] = uniqueZ[iz]
    #             print(ix,iy,iz)
                ix = ix + 1
                iTotal = iTotal+1
            ix = 0
            iy = iy + 1
        iy = 0
        iz = iz + 1

    return CellCentre

In [3]:
def getFieldFromDfColName(CellID,df,colName):
    field = np.zeros(len(CellID))
    i=0
    for c in CellID:
#         print(c)
        if (c >= 0):
            field[i] = df[colName][c]
        i = i+1
    return field

In [4]:
def getCellIDorder(CellCentre,df,tol):
    CellIDcol = -1*np.ones(len(CellCentre),dtype=int)
    i = 0
    for c in CellIDcol:
#     for c in range(0,10):
        res= df[\
                ( abs(df["x-coordinate"]-CellCentre[i,0]) < tol ) \
            &   ( abs(df["y-coordinate"]-CellCentre[i,1]) < tol ) \
            &   ( abs(df["z-coordinate"]-CellCentre[i,2]) < tol ) \
            ]

        if len(res) > 0:    
            valIndex = int(res["cellnumber"].values[0])
            val = valIndex - 1

    #         T[i] = df["total-temperature"][val]
    #         VoF[i] = df["phase-2-vof"][val]
            CellIDcol[i] = val
    #     else:
    #         print("Cell at ", CellCentre[i], " not found" )
#         print("i = " + str(i)  + " C: " + str(CellCentre[i]))
#         print("\n" + str(val) + " Corig: " + str(df["x-coordinate"][val]) + " " + str(df["y-coordinate"][val])+ " " +str(df["z-coordinate"][val]) + "\n")

        i = i+1
    return CellIDcol

In [5]:
def parseCSVtoOpenFOAM(CasePath,Time,csvRelPath,cellPosName,cellPosFactor,cellPosTol,TnameIn,TnameOut,VoFnameIn,VoFnameOut):
    df = pd.read_csv(CasePath+csvRelPath,skipinitialspace=True)
    CellCentre, boundaryStr = ReadVectorFieldValues(CasePath,Time,cellPosName)
    CellCentre = CellCentre/cellPosFactor
    CellID = getCellIDorder(CellCentre,df,cellPosTol)
    
    T = getFieldFromDfColName(CellID,df,TnameIn)
    VoF = getFieldFromDfColName(CellID,df,VoFnameIn)
    
    writeField('scalar',CasePath,Time,TnameOut,T,boundaryStr)
    writeField('scalar',CasePath,Time,VoFnameOut,VoF,boundaryStr)

In [6]:
# Input Parameters, could be read from input arguments
CasePath = "/Users/dd/foam/dockerShared/cafoamrepo/run/NEWAM/Xin/testCase/"
Time="0"

csvRelPath="XinTest"

cellPosName="CellPos"
cellPosFactor=1000
cellPosTol=1e-12

TnameIn="total-temperature"
TnameOut="T"

VoFnameIn="phase-2-vof"
VoFnameOut="alpha.material"

parseCSVtoOpenFOAM(CasePath,Time,csvRelPath,cellPosName,cellPosFactor,cellPosTol,TnameIn,TnameOut,VoFnameIn,VoFnameOut)